<a href="https://colab.research.google.com/github/Shaheen002/Health-Vision/blob/main/docs/examples/rag_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/docling-project/docling/blob/main/docs/examples/rag_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG with LangChain

| Step | Tech | Execution |
| --- | --- | --- |
| Embedding | Hugging Face / Sentence Transformers | 💻 Local |
| Vector store | Milvus | 💻 Local |
| Gen AI | Hugging Face Inference API | 🌐 Remote |

This example leverages the
[LangChain Docling integration](../../integrations/langchain/), along with a Milvus
vector store, as well as sentence-transformers embeddings.

The presented `DoclingLoader` component enables you to:
- use various document types in your LLM applications with ease and speed, and
- leverage Docling's rich format for advanced, document-native grounding.

`DoclingLoader` supports two different export modes:
- `ExportType.MARKDOWN`: if you want to capture each input document as a separate
  LangChain document, or
- `ExportType.DOC_CHUNKS` (default): if you want to have each input document chunked and
  to then capture each individual chunk as a separate LangChain document downstream.

The example allows exploring both modes via parameter `EXPORT_TYPE`; depending on the
value set, the example pipeline is then set up accordingly.

## Setup

- 👉 For best conversion speed, use GPU acceleration whenever available; e.g. if running on Colab, use GPU-enabled runtime.
- Notebook uses HuggingFace's Inference API; for increased LLM quota, token can be provided via env var `HF_TOKEN`.
- Requirements can be installed as shown below (`--no-warn-conflicts` meant for Colab's pre-populated Python env; feel free to remove for stricter usage):

In [2]:
%pip install -q --progress-bar off --no-warn-conflicts langchain-docling langchain-core langchain-huggingface langchain_milvus langchain python-dotenv

  Preparing metadata (setup.py) ... done


In [3]:
DOC_SOURCE = "https://arxiv.org/pdf/2303.08518v4"

In [4]:
from docling.document_converter import DocumentConverter

doc = DocumentConverter().convert(source=DOC_SOURCE).document

[INFO] 2025-12-15 06:19:47,471 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 06:19:47,476 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2025-12-15 06:19:47,480 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.4.0/torch/PP-OCRv4/det/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 06:19:48,612 [RapidOCR] download_file.py:82: Download size: 13.83MB
[INFO] 2025-12-15 06:19:48,816 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 06:19:48,817 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 06:19:49,293 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 06:19:49,294 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2025-12-15 06:19:49,296 [RapidOCR] download_file.py:68: Initiat

In [9]:
"""
Docling Formula Text Mapper

This module handles the extraction of formula content from the 'orig' field
and copies it to the 'text' field in FormulaItem objects from Docling's
document converter output.

Logic:
1. Parse the extracted Docling document schema
2. Iterate through all text items in the body
3. When a FormulaItem is found with 'orig' field populated but 'text' empty
4. Copy the content from 'orig' to 'text' field
5. Return the modified schema
"""

from typing import Dict, Any, List, Optional
import copy


class DoclingFormulaMapper:
    """
    Maps formula content from 'orig' field to 'text' field in Docling documents.
    """

    def __init__(self):
        self.formulas_processed = 0
        self.formulas_skipped = 0

    def process_formula_item(self, formula_item: Dict[str, Any]) -> Dict[str, Any]:
        """
        Process a single formula item and copy orig to text if needed.

        Args:
            formula_item: Dictionary containing formula item data

        Returns:
            Modified formula item with text field populated
        """
        # Check if this is a FormulaItem
        if not isinstance(formula_item, dict):
            return formula_item

        # Check if it has the label indicating it's a formula
        if formula_item.get("label") != "formula":
            return formula_item

        # Get orig and text fields
        orig_content = formula_item.get("orig", "")
        text_content = formula_item.get("text", "")

        # If orig has content but text is empty, copy orig to text
        if orig_content and not text_content:
            formula_item["text"] = orig_content
            self.formulas_processed += 1

            return formula_item
        elif orig_content and text_content:
            # Already has text, skip
            self.formulas_skipped += 1
            return formula_item
        else:
            # No orig content found
            self.formulas_skipped += 1
            return formula_item

    def process_texts_list(self, texts: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Process all text items and map formulas.

        Args:
            texts: List of text item dictionaries from Docling schema

        Returns:
            List of processed text items with formulas mapped
        """
        processed_texts = []

        for text_item in texts:
            processed_item = self.process_formula_item(text_item)
            processed_texts.append(processed_item)

        return processed_texts

    def process_document(self, docling_document_dict: Dict[str, Any]) -> Dict[str, Any]:
        """
        Process the entire Docling document and map all formulas.

        Args:
            docling_document_dict: Dictionary representation of the Docling document

        Returns:
            Modified document dictionary with all formulas mapped
        """
        # Create a deep copy to avoid modifying the original
        document = copy.deepcopy(docling_document_dict)

        # Extract texts from the body
        if "texts" in document and isinstance(document["texts"], list):
            document["texts"] = self.process_texts_list(document["texts"])

        return document

    def get_statistics(self) -> Dict[str, int]:
        """
        Get statistics about the formula processing.

        Returns:
            Dictionary with processing statistics
        """
        return {
            "formulas_processed": self.formulas_processed,
            "formulas_skipped": self.formulas_skipped,
            "total_processed": self.formulas_processed + self.formulas_skipped,
        }


def extract_and_map_formulas(doc_dict: Dict[str, Any]) -> Dict[str, Any]:
    """
    Standalone function to extract and map formulas in a document.

    Args:
        doc_dict: Dictionary representation of the Docling document

    Returns:
        Modified document with formulas mapped
    """
    mapper = DoclingFormulaMapper()
    processed_doc = mapper.process_document(doc_dict)

    print("\n=== Formula Mapping Statistics ===")
    stats = mapper.get_statistics()
    for key, value in stats.items():
        print(f"{key}: {value}")

    return processed_doc


# ============================================================================
# USAGE EXAMPLE
# ============================================================================

if __name__ == "__main__":
    from docling.document_converter import DocumentConverter

    # Example: Convert a PDF document
    PDF_PATH = "https://arxiv.org/pdf/2212.10496v1"

    try:
        # Convert document using Docling
        converter = DocumentConverter()
        doc = converter.convert(source=PDF_PATH).document

        # Convert Docling document to dictionary representation
        # (This depends on Docling's export capabilities)
        doc_dict = doc.model_dump()  # or similar method depending on Docling version

        # Process formulas
        processed_doc = extract_and_map_formulas(doc_dict)

        # Verify the changes
        print("\n=== Example Formula Items ===")
        if "texts" in processed_doc:
            formula_items = [
                item for item in processed_doc["texts"]
                if item.get("label") == "formula"
            ]
            for idx, formula in enumerate(formula_items[:3]):  # Show first 3
                print(f"\nFormula {idx + 1}:")
                print(f"  Original (orig): {formula.get('orig', 'N/A')}")
                print(f"  Text field (text): {formula.get('text', 'N/A')}")

    except FileNotFoundError:
        print(f"Document not found: {PDF_PATH}")
    except Exception as e:
        print(f"Error processing document: {e}")
        print("\nTo use this script:")
        print("1. Install Docling: pip install docling")
        print("2. Replace PDF_PATH with your actual document path")
        print("3. Run the script")

[INFO] 2025-12-15 10:10:11,971 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 10:10:11,974 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2025-12-15 10:10:12,045 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 10:10:12,046 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 10:10:12,556 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 10:10:12,557 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2025-12-15 10:10:12,563 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 10:10:12,564 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-1


=== Formula Mapping Statistics ===
formulas_processed: 8
formulas_skipped: 0
total_processed: 8

=== Example Formula Items ===

Formula 1:
  Original (orig): sim ( q , d ) = 〈 enc q ( q ) , enc d ( d ) 〉 = 〈 v q , v d 〉 (1)
  Text field (text): sim ( q , d ) = 〈 enc q ( q ) , enc d ( d ) 〉 = 〈 v q , v d 〉 (1)

Formula 2:
  Original (orig): f = enc d = enccon (2)
  Text field (text): f = enc d = enccon (2)

Formula 3:
  Original (orig): v d = f ( d ) ∀ d ∈ D 1 ∪ D 2 ∪ ... ∪ D L (3)
  Text field (text): v d = f ( d ) ∀ d ∈ D 1 ∪ D 2 ∪ ... ∪ D L (3)


In [98]:
from docling.document_converter import DocumentConverter
from docling.datamodel.document import DoclingDocument

class DoclingFormulaMapper:
    def __init__(self):
        self.processed = 0
        self.skipped = 0

    def process(self, doc: DoclingDocument) -> DoclingDocument:
        for item in doc.texts:
            if item.label == "formula":
                if item.orig and not item.text:
                    item.text = item.orig
                    self.processed += 1
                else:
                    self.skipped += 1
        return doc

doc = DocumentConverter().convert(source="https://arxiv.org/pdf/2212.10496").document

mapper = DoclingFormulaMapper()
processed_doc = mapper.process(doc)

print("Formulas processed:", mapper.processed)

from docling.chunking import HybridChunker

chunker = HybridChunker(
    max_tokens=3000,        # IMPORTANT (prevents 512 overflow)
)

chunk_iter = chunker.chunk(dl_doc=processed_doc)



[INFO] 2025-12-15 12:00:28,602 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 12:00:28,605 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2025-12-15 12:00:28,703 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 12:00:28,707 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 12:00:29,729 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 12:00:29,735 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2025-12-15 12:00:29,746 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 12:00:29,747 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-1

Formulas processed: 8


Token indices sequence length is longer than the specified maximum sequence length for this model (778 > 512). Running this sequence through the model will result in indexing errors


In [99]:
for i, chunk in enumerate(chunk_iter):
    enriched_text = chunker.contextualize(chunk)
    print(f"=== Chunk {i} ===")
    print(enriched_text)


=== Chunk 0 ===
Precise Zero-Shot Dense Retrieval without Relevance Labels
Luyu Gao ∗ † Xueguang Ma ∗ ‡ Jimmy Lin ‡ Jamie Callan †
†
Language Technologies Institute, Carnegie Mellon University ‡ David R. Cheriton School of Computer Science, University of Waterloo {luyug, callan}@cs.cmu.edu, {x93ma, jimmylin}@uwaterloo.ca
=== Chunk 1 ===
Abstract
While dense retrieval has been shown effective and efficient across tasks and languages, it remains difficult to create effective fully zero-shot dense retrieval systems when no relevance label is available. In this paper, we recognize the difficulty of zero-shot learning and encoding relevance. Instead, we propose to pivot through Hypothetical Document Embeddings ( HyDE ). Given a query, HyDE first zero-shot instructs an instruction-following language model (e.g. InstructGPT ) to generate a hypothetical document. The document captures relevance patterns but is unreal and may contain false details. Then, an unsupervised contrastively learned en

In [20]:
for i, chunk in enumerate(chunk_iter):
    print(f"=== {i} ===")
    print(f"chunk.text:\n{f'{chunk.text[:300]}…'!r}")

    enriched_text = chunker.contextualize(chunk=chunk)
    print(f"chunker.contextualize(chunk):\n{f'{enriched_text[:300]}…'!r}")

    print()

In [90]:
import os
from pathlib import Path
from tempfile import mkdtemp

from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_docling.loader import ExportType


def _get_env_from_colab_or_os(key):
    try:
        from google.colab import userdata

        try:
            return userdata.get(key)
        except userdata.SecretNotFoundError:
            pass
    except ImportError:
        pass
    return os.getenv(key)


load_dotenv()

# https://github.com/huggingface/transformers/issues/5486:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

HF_TOKEN = _get_env_from_colab_or_os("HF_TOKEN")
FILE_PATH = ["https://arxiv.org/pdf/2212.10496"]  # Docling Technical Report
EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
GEN_MODEL_ID = "mistralai/Mixtral-8x7B-Instruct-v0.1"
EXPORT_TYPE = ExportType.DOC_CHUNKS
QUESTION = "Which are the main AI models in Docling?"
PROMPT = PromptTemplate.from_template(
    "Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {input}\nAnswer:\n",
)
TOP_K = 7
MILVUS_URI = str(Path(mkdtemp()) / "docling.db")

## Document loading

Now we can instantiate our loader and load documents.

In [41]:
from langchain_docling import DoclingLoader

from docling.chunking import HybridChunker

loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
    chunker=HybridChunker(tokenizer=EMBED_MODEL_ID),
)

docs = loader.load()

[INFO] 2025-12-15 10:34:39,499 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 10:34:39,501 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2025-12-15 10:34:39,581 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 10:34:39,583 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-12-15 10:34:39,985 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-15 10:34:39,987 [RapidOCR] device_config.py:57: Using GPU device with ID: 0
[INFO] 2025-12-15 10:34:39,994 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-15 10:34:39,996 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-12-1

> Note: a message saying `"Token indices sequence length is longer than the specified
maximum sequence length..."` can be ignored in this case — details
[here](https://github.com/docling-project/docling-core/issues/119#issuecomment-2577418826).

In [35]:
docs

[Document(metadata={'source': 'https://arxiv.org/pdf/2408.09869', 'dl_meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/3', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 113.643, 't': 481.532, 'r': 498.359, 'b': 439.849, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 295]}]}, {'self_ref': '#/texts/4', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 249.283, 't': 427.545, 'r': 362.717, 'b': 408.084, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 50]}]}], 'headings': ['Version 1.0'], 'origin': {'mimetype': 'application/pdf', 'binary_hash': 11465328351749295394, 'filename': '2408.09869v5.pdf'}}}, page_content='Version 1.0\nChristoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbaue

Determining the splits:

In [50]:
if EXPORT_TYPE == ExportType.DOC_CHUNKS:
    splits = docs
elif EXPORT_TYPE == ExportType.MARKDOWN:
    from langchain_text_splitters import MarkdownHeaderTextSplitter

    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header_1"),
            ("##", "Header_2"),
            ("###", "Header_3"),
        ],
    )
    splits = [split for doc in docs for split in splitter.split_text(doc.page_content)]
else:
    raise ValueError(f"Unexpected export type: {EXPORT_TYPE}")

Inspecting some sample splits:

In [51]:
print(splits)

[Document(metadata={'source': 'https://arxiv.org/pdf/2212.10496', 'dl_meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/2', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 138.755, 't': 724.9550146484376, 'r': 459.011, 'b': 712.6740146484375, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 55]}]}, {'self_ref': '#/texts/3', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 149.327, 't': 710.8980146484375, 'r': 153.09, 'b': 704.0460146484376, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 1]}]}, {'self_ref': '#/texts/4', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 1, 'bbox': {'l': 125.339, 't': 708.8980146484375, 'r': 472.929, 'b': 670.6310146484375, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 189]}]

In [52]:
for d in splits[:]:
    print(f"- {d.page_content=}")
print("...")

- d.page_content='Precise Zero-Shot Dense Retrieval without Relevance Labels\nLuyu Gao ∗ † Xueguang Ma ∗ ‡ Jimmy Lin ‡ Jamie Callan †\n†\nLanguage Technologies Institute, Carnegie Mellon University ‡ David R. Cheriton School of Computer Science, University of Waterloo {luyug, callan}@cs.cmu.edu, {x93ma, jimmylin}@uwaterloo.ca'
- d.page_content="Abstract\nWhile dense retrieval has been shown effective and efficient across tasks and languages, it remains difficult to create effective fully zero-shot dense retrieval systems when no relevance label is available. In this paper, we recognize the difficulty of zero-shot learning and encoding relevance. Instead, we propose to pivot through Hypothetical Document Embeddings ( HyDE ). Given a query, HyDE first zero-shot instructs an instruction-following language model (e.g. InstructGPT ) to generate a hypothetical document. The document captures relevance patterns but is unreal and may contain false details. Then, an unsupervised contrastively l

## Ingestion

In [54]:
pip install milvus_lite langchain_classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.1 MB/s eta 0:00:00


In [55]:
import json
from pathlib import Path
from tempfile import mkdtemp

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_milvus import Milvus

embedding = HuggingFaceEmbeddings(model_name=EMBED_MODEL_ID)


milvus_uri = str(Path(mkdtemp()) / "docling.db")  # or set as needed
vectorstore = Milvus.from_documents(
    documents=splits,
    embedding=embedding,
    collection_name="docling_demo",
    connection_args={"uri": milvus_uri},
    index_params={"index_type": "FLAT"},
    drop_old=True,
)

## RAG

In [62]:
%pip install -q openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 3.3 MB/s eta 0:00:00


In [91]:
# --- Replacement for HuggingFaceEndpoint ---

from langchain_openai import ChatOpenAI
# The same settings you used for LlamaIndex will apply here
HUGGINGFACE_OPENAI_BASE_URL = "https://router.huggingface.co/hf-inference/models/HuggingFaceTB/SmolLM3-3B/v1"
GEN_MODEL_ID_FOR_OPENAI = "HuggingFaceTB/SmolLM3-3B"

llm = ChatOpenAI(
    model=GEN_MODEL_ID_FOR_OPENAI, # The model name (required by the router)
    api_key=HF_TOKEN,             # Your Hugging Face token
    base_url=HUGGINGFACE_OPENAI_BASE_URL, # The custom Hugging Face router URL
)

# --- The rest of your RAG chain remains the same ---
retriever = vectorstore.as_retriever(search_kwargs={"k": TOP_K})
question_answer_chain = create_stuff_documents_chain(llm, PROMPT) # 'llm' is now ChatOpenAI
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
def clip_text(text, threshold=1000):
# ... [rest of your code]
    return f"{text[:threshold]}..." if len(text) > threshold else text

In [96]:
QUESTION = "What does 2 related works speaks about instruciton following language models?"

In [97]:
question_answer_chain = create_stuff_documents_chain(llm, PROMPT)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
resp_dict = rag_chain.invoke({"input": QUESTION})

clipped_answer = clip_text(resp_dict["answer"],)
print(f"Question:\n{resp_dict['input']}\n\nAnswer:\n{clipped_answer}")
for i, doc in enumerate(resp_dict["context"]):
    print()
    print(f"Source {i + 1}:")
    print(f"  text: {json.dumps(clip_text(doc.page_content,))}")
    for key in doc.metadata:
        if key != "pk":
            val = doc.metadata.get(key)
            clipped_val = clip_text(val) if isinstance(val, str) else val
            print(f"  {key}: {clipped_val}")

APIStatusError: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}